In [1]:
# Dependencies
from bs4 import BeautifulSoup
from splinter import Browser
import pandas as pd
import requests
import pymongo

In [2]:
# https://splinter.readthedocs.io/en/latest/drivers/chrome.html
!which chromedriver

/usr/local/bin/chromedriver


In [3]:
#Choose the executable path to driver
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [4]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [5]:
# Define database and collection
db = client.missiontomars_db
collection = db.items

### NASA Mars News

In [6]:
# URL of page to be scraped
url = 'https://mars.nasa.gov/news/'
browser.visit(url)

# Retrieve page with the requests module
response = requests.get(url)
# Create BeautifulSoup object; parse with 'lxml'
soup = BeautifulSoup(response.text, 'lxml')

In [7]:
# results are returned as an iterable list
results = soup.find('div', class_='features')


# Identify and return title of listing
news_title = results.find('div', class_='content_title').a.text
# Identify and return paragraph
news_p = results.find('div', class_='rollover_description_inner').text 

      
# Print results
print(news_title)
print(news_p)      


NASA's InSight Places First Instrument on Mars


In deploying its first instrument onto the surface of Mars, the lander completes a major mission milestone.



### JPL Mars Space Images - Featured Image

In [8]:
#Visit Mars Space Images through splinter
space_url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(space_url)

In [9]:
# Locate full image and click
full_image_elem = browser.find_by_id('full_image')
full_image_elem.click()

In [10]:
# Find button obtain additional information
browser.is_element_present_by_text('more info', wait_time=1)
more_info_elem = browser.find_link_by_partial_text('more info')

In [11]:
# Click more info button
more_info_elem.click()

In [12]:
# HTML object 
html = browser.html
# Create BeautifulSoup object; parse with 'lxml'
img_soup = BeautifulSoup(html, 'html.parser')

In [13]:
# Image URL 
main_image_rel = img_soup.select_one('figure.lede a img').get("src")
main_image_rel

'/spaceimages/images/largesize/PIA09113_hires.jpg'

In [28]:
# Complete url for image
featured_image_url = f"https://www.jpl.nasa.gov{main_image_rel}"
featured_image_url

'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA09113_hires.jpg'

### Mars Weather

In [15]:
# Visit Mars Twitter
weather_url = 'https://twitter.com/marswxreport?lang=en'
browser.visit(weather_url)

# Retrieve page with the requests module
response = requests.get(weather_url)
# Create BeautifulSoup object; parse with 'lxml'
weather_soup = BeautifulSoup(response.text, 'lxml')

In [16]:
# Find all elements that contain tweets
mars_tweets = weather_soup.find_all('div', class_='js-tweet-text-container')

# Loop to retrieve Sol & Pressure
for tweet in mars_tweets:
    mars_weather = tweet.find('p').text
    if 'Sol' and 'pressure' in mars_weather:
        print(f' Mars weather is {mars_weather}')
        break
    else:
        pass    

 Mars weather is Sol 2271 (2018-12-26), high -7C/19F, low -70C/-93F, pressure at 8.31 hPa, daylight 06:41-18:54


### Mars Facts

In [32]:
# Visit Mars Facts
facts_url = 'https://space-facts.com/mars/'   

# Read facts 
mars_facts = pd.read_html(facts_url)

# Create df to receive Mars Facts
mars = mars_facts[0]

# Assign columns and index
mars.columns = ['Description', 'Value']
mars.set_index('Description', inplace=True)

# Save html code
mars.to_html()
data = mars.to_dict(orient='records')

#show df
mars

,Value
Description,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.42 x 10^23 kg (10.7% Earth)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.52 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-153 to 20 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


### Mars Hemisphere

In [18]:
# Visit html & find list of links for hemisphere
hemisphere_url = 'http://web.archive.org/web/20181114171728/https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(hemisphere_url)
links = browser.find_by_css("a.product-item h3")


In [29]:
# loop through links
hemisphere_image_urls =[]
for i in range(len(links)):
    browser.visit(hemisphere_url)
    #hemi = {}
    browser.find_by_css("a.product-item h3")[i].click()
    # Next, we find the Sample image anchor tag and extract the href
    sample_elem = browser.find_link_by_text('Sample').first
    title = browser.find_by_css("h2.title").text
    img_url = sample_elem['href']
    
    hemisphere_image_urls.append({"title" : title, "img_url": img_url})
    
hemisphere_image_urls

[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'http://web.archive.org/web/20181114182238/http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'http://web.archive.org/web/20181114182242/http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'http://web.archive.org/web/20181114182245/http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'http://web.archive.org/web/20181114182248/http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]